# Simple QA to PostgreSQL

This script is to move the data from files to postgreSQL to help analyze.

In [15]:
import os
import psycopg2
import psycopg2.extras

# Load .env file
pass_ = {}
for line in open('../../.pass'):
    split = line.strip().split('=')
    pass_[split[0]] = split[1]

# Connect    
connection = psycopg2.connect(dbname=pass_['DB_NAME'],
                              port=pass_['DB_PORT'],
                              user=pass_['DB_USER'],
                              host=pass_['DB_HOST'],
                              password=pass_['DB_PASS'])
cursor = connection.cursor()

## Index FB5M KG

In [2]:
fb_kg = '../../data/SimpleQuestions_v2/freebase-subsets/freebase-FB5M.txt'

In [3]:
cursor.execute("""
    CREATE TABLE fb_kg
        (object_mid varchar NOT NULL,
        relation varchar NOT NULL,
        subject_mid varchar NOT NULL,
        PRIMARY KEY(object_mid, relation, subject_mid));
        CREATE INDEX relation_index ON fb_kg (relation);
        CREATE INDEX object_mid_index ON fb_kg (object_mid);
        CREATE INDEX subject_mid_index ON fb_kg (subject_mid);""")

In [4]:
from tqdm import tqdm_notebook

chunk_size = 20000


def insert_chunk(rows):
    insert_query = 'INSERT INTO fb_kg (object_mid, relation, subject_mid) VALUES %s ON CONFLICT DO NOTHING;'
    psycopg2.extras.execute_values(
        cursor, insert_query, rows, template=None, page_size=100
    )
    

rows = []
for line in tqdm_notebook(open(fb_kg, 'r'), total=12010500):
    # Build Chunks
    split = line.split('\t')
    assert len(split) == 3, 'Malformed row'
    subject = split[0].replace('www.freebase.com/m/', '').strip()
    property_ = split[1].replace('www.freebase.com/', '').strip()
    objects = [url.replace('www.freebase.com/m/', '').strip() for url in split[2].split()]
    rows.extend([tuple([str(object_), str(property_), str(subject)]) for object_ in objects])

    # Insert Chunk
    if len(rows) > chunk_size:
        insert_chunk(rows)
        rows = []
        
insert_chunk(rows)

HBox(children=(IntProgress(value=0, max=12010500), HTML(value='')))

In [5]:
connection.commit()

#### Some quick numbers:

- Total 12,010,500 FB5M tuples. Coraborated by https://research.fb.com/wp-content/uploads/2016/11/large-scale_simple_question_answering_with_memory_networks.pdf?
- Subject Unique MIDs ~ 3,929,894
- Subject MIDs ~ 46,424,988
- Object Unique MIDs ~ 1,972,702
- Object MIDs ~ 12,010,500. Coraborated by https://research.fb.com/wp-content/uploads/2016/11/large-scale_simple_question_answering_with_memory_networks.pdf?
- Total Unique MIDs ~ 3,988,105 ~ Coraborated by https://github.com/zihangdai/CFO
- Total MIDs ~ 58,435,488
- Total Relations ~ 7,523. Coraborated by https://research.fb.com/wp-content/uploads/2016/11/large-scale_simple_question_answering_with_memory_networks.pdf?
- Total Unique Facts ~ 17,872,174. Coraborated by https://github.com/zihangdai/CFO 


## Index FB5M Mid to Names

In [6]:
fb_name = '../../data/FB5M.name.txt'

In [7]:
cursor.execute("""
    CREATE TABLE fb_name
        (mid varchar NOT NULL,
        relation varchar NOT NULL,
        alias varchar NOT NULL,
        PRIMARY KEY(mid, relation, alias));
    CREATE INDEX alias_index ON fb_name (alias);""")

In [8]:
chunk_size = 10000


def insert_chunk(rows):
    insert_query = 'INSERT INTO fb_name (mid, relation, alias) VALUES %s ON CONFLICT DO NOTHING;'
    psycopg2.extras.execute_values(
        cursor, insert_query, rows, template=None, page_size=100
    )
    

rows = []
for line in tqdm_notebook(open(fb_name, 'r'), total=5507279):
    # Build Chunks
    split = line.strip('.\n').strip().split('\t')
    assert len(split) == 3, 'Malformed row'
    mid = split[0].strip().replace('<fb:m.', '').rstrip('>')
    relation = split[1].strip().replace('<fb:', '').rstrip('>')
    alias = split[2].strip().strip('\'').strip('"')
    rows.append(tuple([mid, relation, alias]))

    # Insert Chunk
    if len(rows) > chunk_size:
        insert_chunk(rows)
        rows = []
        
insert_chunk(rows)

HBox(children=(IntProgress(value=0, max=5507279), HTML(value='')))

In [10]:
connection.commit()

## Index FB5M Mid to Type

In [12]:
fb_type = '../../data/FB5M.type.txt'

In [16]:
cursor.execute("""
    CREATE TABLE fb_type
        (mid varchar NOT NULL,
        relation varchar NOT NULL,
        type varchar NOT NULL,
        PRIMARY KEY(mid, relation, type))""")

In [17]:
chunk_size = 10000


def insert_chunk(rows):
    insert_query = 'INSERT INTO fb_type (mid, relation, type) VALUES %s ON CONFLICT DO NOTHING;'
    psycopg2.extras.execute_values(
        cursor, insert_query, rows, template=None, page_size=100
    )
    

rows = []
for line in tqdm_notebook(open(fb_type, 'r'), total=23649940):
    # Build Chunks
    split = line.strip('.\n').strip().split('\t')
    assert len(split) == 3, 'Malformed row'
    mid = split[0].strip().replace('<fb:m.', '').strip('>')
    relation = split[1].strip().replace('<fb:', '').strip('>')
    type_ = split[2].strip().replace('<fb:', '').strip('>')
    rows.append(tuple([mid, relation, type_]))

    # Insert Chunk
    if len(rows) > chunk_size:
        insert_chunk(rows)
        rows = []

insert_chunk(rows)

HBox(children=(IntProgress(value=0, max=23649940), HTML(value='')))

In [19]:
connection.commit()

## Shut Down Connection

In [12]:
cursor.close()
connection.close()